In [ ]:
# default_exp metrics

# Custom losses and metrics

In [ ]:
# export
from drone_detector.imports import *
from fastai.learner import Metric
from fastai.torch_core import *
from fastai.metrics import *
from fastai.losses import BaseLoss
import torch
import torch.nn.functional as F

/home/mayrajeo/miniconda3/envs/enveco/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729062494/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [ ]:
# export
class DiceMulti(Metric):
    "Averaged Dice metric (Macro F1) for multiclass target in segmentation. Newer fastai-version has this implemented"
    def __init__(self, axis=1): self.axis = axis
    def reset(self): self.inter,self.union = {},{}
    def accumulate(self, learn):
        pred,targ = flatten_check(learn.pred.argmax(dim=self.axis), learn.y)
        for c in range(learn.pred.shape[self.axis]):
            p = torch.where(pred == c, 1, 0)
            t = torch.where(targ == c, 1, 0)
            c_inter = (p*t).float().sum().item()
            c_union = (p+t).float().sum().item()
            if c in self.inter:
                self.inter[c] += c_inter
                self.union[c] += c_union
            else:
                self.inter[c] = c_inter
                self.union[c] = c_union

    @property
    def value(self):
        binary_dice_scores = np.array([])
        for c in self.inter:
            binary_dice_scores = np.append(binary_dice_scores, 2.*self.inter[c]/self.union[c] if self.union[c] > 0 else np.nan)
        return np.nanmean(binary_dice_scores)

In [ ]:
#export

def adjusted_R2Score(r2_score, n, k):
    "Calculates adjusted_R2Score based on r2_score, number of observations (n) and number of predictor variables(k)"
    return 1 - (((n-1)/(n-k-1)) * (1 - r2_score))


In [ ]:
#export

def _rrmse(inp, targ):
    "RMSE normalized with mean of the target"
    return torch.sqrt(F.mse_loss(inp, targ)) / targ.mean() * 100

rrmse = AccumMetric(_rrmse)
rrmse.__doc__ = "Target mean weighted rmse"

In [ ]:
#export
def _bias(inp, targ):
    "Average bias of predictions"
    inp, targ = flatten_check(inp, targ)
    return (inp - targ).sum() / len(targ)

bias = AccumMetric(_bias)
bias.__doc__ = "Average bias of predictions"

In [ ]:
#export
def _bias_pct(inp, targ):
    "Mean weighted bias"
    inp, targ = flatten_check(inp, targ)
    return 100 * ((inp-targ).sum()/len(targ)) / targ.mean()

bias_pct = AccumMetric(_bias_pct)
bias_pct.__doc__ = 'Mean weighted bias'